# Pooled dataset
- split main dataset into train and val
- find our 10 classes data
- save as processed_dataset.csv

In [ ]:
# === Cell 1: Imports & Global Config ===
import os, re, json, random, math
from pathlib import Path
import numpy as np
import pandas as pd
from collections import defaultdict, Counter

# Reproducibility
RNG_SEED = 42
random.seed(RNG_SEED)
np.random.seed(RNG_SEED)

# File paths
DATASET_CSV = "..\data\pooled_images.csv" # pooled dataset
LABELS_CSV  = "..\data\images.csv"        # our dataset
SUGGESTED_MAP_JSON = "suggested_mapping.json"
PROCESSED_CSV = "..\data\processed_dataset.csv"

print("Environment ready ✅")

Environment ready ✅


Load .csv files which contains metadata for images dataset and pooled dataset

In [3]:
# === Cell 2: Load Dataset & Labels ===
def robust_read_csv(path):
    try:
        return pd.read_csv(path, low_memory=False)
    except Exception:
        return pd.read_csv(path, encoding='latin1', low_memory=False)

df = robust_read_csv(DATASET_CSV)
df_labels = robust_read_csv(LABELS_CSV)

# dropping unnecessary columns
df.drop(columns=['Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20', 'Unnamed: 21', 'Unnamed: 22', 'Unnamed: 23'],inplace=True)
print("Dataset columns:", df.columns.tolist())
print("Labels columns:", df_labels.columns.tolist())
df.head()


Dataset columns: ['image_path', 'class_label', 'attributes', 'caption', 'instance_id']
Labels columns: ['image_path', 'class_label', 'attributes', 'caption', 'split', 'instance_id']


,image_path,class_label,attributes,caption,instance_id
0,images\team14_wrist_watch (1).jpg,clothing_wrist_watch,color:black;material:unknown;condition:new;siz...,new small black wrist watch face,Team14_BlackWatch_A
1,images\team14_wrist_watch (2).jpg,clothing_wrist_watch,color:black;material:plastic;condition:new;siz...,new small black plastic watch from side,Team14_BlackWatch_B
2,images\team14_wrist_watch (3).jpg,clothing_wrist_watch,color:yellow;material:metal;condition:new;size...,new small yellow metal watch lying straight,Team14_YellowWatch_A
3,images\team14_wrist_watch (4).jpg,clothing_wrist_watch,color:black;material:unknown;condition:new;siz...,new small black leather watch,Team14_BlackWatch_C
4,images\team14_wrist_watch (5).jpg,clothing_wrist_watch,color:black;material:unknown;condition:new;siz...,new small black wrist watch face,Team14_BlackWatch_D


In [4]:
# === Cell 3: Detect the class/label columns ===
def detect_class_col(df):
    for c in df.columns:
        if c.lower() in ['class','label','class_label','classname','category']:
            return c
    for c in df.columns:
        if df[c].dtype == object and df[c].nunique() < 1000:
            return c
    return df.columns[0]

dataset_class_col = detect_class_col(df)
labels_class_col  = detect_class_col(df_labels)

print("Dataset class column:", dataset_class_col)
print("Labels class column:", labels_class_col)


Dataset class column: class_label
Labels class column: class_label


In [5]:
# === Cell 4: Inspect unique class names ===
df[dataset_class_col] = df[dataset_class_col].astype(str).str.strip()
df_labels[labels_class_col] = df_labels[labels_class_col].astype(str).str.strip()

unique_dataset_classes = sorted(df[dataset_class_col].unique().tolist())
canonical_classes = sorted(df_labels[labels_class_col].unique().tolist())

print(f"Found {len(unique_dataset_classes)} unique dataset entries.")
print("Sample dataset classes:", unique_dataset_classes[:30])
print("\nCanonical 10 classes from labels.csv:", canonical_classes)


Found 197 unique dataset entries.
Sample dataset classes: ['Clothing_wallet', 'Cosmetics_lipstick', 'Cup', 'Earpods_headphones', 'Electronics_Accessories_mouse', 'Footwear_Sneakers', 'Medical_pill_strip', 'Personal_Care_shampoo_bottle', 'Shampoo_bottle', 'Stationery_Pen', 'Stationery_notebook', 'Tableware_Water_Bottle', 'Tools_screwdriver', 'Travel_backpack', 'Travel_bagpack', 'backpack', 'bathroom_bucket', 'bathroom_comb', 'bathroom_handwashing', 'bathroom_loofah', 'bathroom_razor', 'bathroom_shampoo_bottle', 'bathroom_soapbox', 'bathroom_tap', 'bathroom_tongue_cleaner', 'bathroom_toothbrush', 'bathroom_toothpaste', 'bathroom_tumbler', 'bottle', 'brush']

Canonical 10 classes from labels.csv: ['Cup', 'calculator', 'cap', 'earphones', 'keyboard', 'mouse', 'scissor', 'sneakers', 'umbrella', 'wristwatch']


In [6]:
pd.DataFrame(unique_dataset_classes)

,0
0,Clothing_wallet
1,Cosmetics_lipstick
2,Cup
3,Earpods_headphones
4,Electronics_Accessories_mouse
...,...
192,water_bottle_7
193,water_bottle_8
194,water_bottle_9
195,wrist_watch


In [8]:
# === Cell 5 (Updated): Smarter class merging into 10 canonical classes ===
import difflib

def normalize_name(s):
    """
    Cleans class names so that variants like 'water_bottle_7' or 'WaterBottle8'
    become 'waterbottle'.
    """
    s = str(s).lower().strip()
    s = re.sub(r'[^a-z0-9]', '', s)         # remove underscores, hyphens, spaces
    s = re.sub(r'\d+$', '', s)              # drop trailing numbers
    return s

# Normalized canonical class names
canonical_classes = sorted(df_labels[labels_class_col].unique().tolist())
canonical_clean = {normalize_name(c): c for c in canonical_classes}

mapping = {}
for u in unique_dataset_classes:
    norm_u = normalize_name(u)
    mapped = None

    # 1️⃣ Exact normalized match
    if norm_u in canonical_clean:
        mapped = canonical_clean[norm_u]
    else:
        # 2️⃣ Substring match (e.g., norm_u contains 'waterbottle')
        for can_clean, can_orig in canonical_clean.items():
            if can_clean in norm_u or norm_u in can_clean:
                mapped = can_orig
                break

    # 3️⃣ Fuzzy match (if still not found)
    if not mapped:
        close = difflib.get_close_matches(norm_u, list(canonical_clean.keys()), n=1, cutoff=0.7)
        if close:
            mapped = canonical_clean[close[0]]

    mapping[u] = mapped

# Save mapping to JSON
with open(SUGGESTED_MAP_JSON, "w") as f:
    json.dump(mapping, f, indent=2)

# Summary
mapped_count = sum(1 for v in mapping.values() if v is not None)
unmapped = [k for k,v in mapping.items() if v is None]
print(f"✅ Auto-mapped {mapped_count} / {len(mapping)} dataset classes to canonical 10 classes.")
if unmapped:
    print("Unmapped examples (inspect manually if needed):", unmapped[:20])
else:
    print("All classes mapped successfully 🎉")


✅ Auto-mapped 29 / 197 dataset classes to canonical 10 classes.
Unmapped examples (inspect manually if needed): ['Clothing_wallet', 'Cosmetics_lipstick', 'Earpods_headphones', 'Medical_pill_strip', 'Personal_Care_shampoo_bottle', 'Shampoo_bottle', 'Stationery_Pen', 'Stationery_notebook', 'Tableware_Water_Bottle', 'Tools_screwdriver', 'Travel_backpack', 'Travel_bagpack', 'backpack', 'bathroom_bucket', 'bathroom_comb', 'bathroom_handwashing', 'bathroom_loofah', 'bathroom_razor', 'bathroom_shampoo_bottle', 'bathroom_soapbox']


In [9]:
# === Cell 6 (updated): Apply mapping, drop 'unknown', split train/val ===
from sklearn.model_selection import train_test_split

df["mapped_class"] = df[dataset_class_col].map(mapping)
df = df[df["mapped_class"].isin(canonical_classes)].reset_index(drop=True)  # ✅ keep only 10 canonical classes

print(f"Kept {len(df)} rows for canonical classes:", sorted(df["mapped_class"].unique()))

def stratified_split(df, col="mapped_class", train_frac=0.8):
    df["split"] = None
    for c, group in df.groupby(col):
        idx = group.index
        if len(idx) > 1:
            train_idx, val_idx = train_test_split(idx, train_size=train_frac, random_state=RNG_SEED)
            df.loc[train_idx, "split"] = "train"
            df.loc[val_idx, "split"] = "val"
        else:
            df.loc[idx, "split"] = "train"
    return df

df = stratified_split(df)
df.to_csv(PROCESSED_CSV, index=False)
print("✅ Processed dataset saved without 'unknown' classes at:", PROCESSED_CSV)
print("Split counts:\n", df["split"].value_counts())
print("Class counts:\n", df["mapped_class"].value_counts())


Kept 3091 rows for canonical classes: ['Cup', 'calculator', 'cap', 'earphones', 'keyboard', 'mouse', 'scissor', 'sneakers', 'umbrella', 'wristwatch']
✅ Processed dataset saved without 'unknown' classes at: ..\data\processed_dataset.csv
Split counts:
 split
train    2471
val       620
Name: count, dtype: int64
Class counts:
 mapped_class
sneakers      600
mouse         556
wristwatch    534
earphones     399
keyboard      320
Cup           200
cap           200
calculator    139
scissor        83
umbrella       60
Name: count, dtype: int64
